In [1]:
# Obtiene una máscara para cada municipio del país.

import os

import rioxarray

import pandas as pd
import numpy as np

import geopandas as gpd
import xarray as xr

In [2]:
vars = ["Tmax", "Tmin", "Tmean", "Pre"]

path_d = "../../../datos/CHIRPS/"
path_r = "../../../resultados/Municipios/"
path_shp = "../../../datos/Municipios/Mapas/mun20gw.shp"

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)

# Se cargan los municipios.
gdf = gpd.read_file( path_shp )

# Se obtiene el contorno de los municipios.
gdf["boundary"] = gdf.boundary

# Renombramos la columna que identifica cada 
# municipio con el valor único de INEGI.
gdf.rename(columns = {
    "CVEGEO": "CVE_INEGI"},
    inplace = True)

In [3]:
# Abrimos el netCDF para el cual se hará la 
# máscara de acuerdo con su resolución.
ds = xr.open_dataset( path_d +
    "/mexico_" + vars[3] + ".1995.nc" )
# Establecemos np.nan como valor sin datos.
ds = ds.where(ds > -9000)
# Se establece el datum de los datos.
ds = ds.rio.write_crs(gdf.crs)
# Se elimina la variable de tiempo.
ds = ds.isel(time = 0).drop("time")

# Se hace un Dataset con unos para todo el país.
mask_0 = ( ds.where(ds.isnull(), 1)
    .expand_dims(dim = "CVE_INEGI") )

# La lista que contra los Datasets la 
# máscara para cada municipio.
mask = []

# Se hace la máscara para cada municipio.
for i in range(gdf.shape[0]):
    mask.append( mask_0.rio.clip(
        gdf[gdf.index == i].geometry,
        gdf.crs, drop=False, invert=False) )

# Para los municipios en los que no cae ningún
# punto, se relajan las condiciones para
# considerar cualquier punto que toque.
for i in range(len(mask)):
    if ( mask[i].where(mask[i] > -9000)
        .drop("spatial_ref")
        .to_dataframe().mean()[0] 
        ) != 1:
        
        mask[i] = mask_0.rio.clip(
        gdf[gdf.index == i].geometry,
        gdf.crs, all_touched = True, 
        drop=False, invert=False)

# Se unen las máscaras para todos los municipios.
mask = ( xr.concat(mask, dim = "CVE_INEGI")
    .drop("spatial_ref")
    .rename(Pre = "mask") )
mask["CVE_INEGI"] = ( gdf["CVE_INEGI"]
    .astype(int).array )
mask = mask.where(mask > -9000)
mask.to_netcdf(path_r + "municipios_mask.nc")